In [60]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

##### The above line is to prevent the kernel from dying and restarting

#### Step 1: Get the emoji package

In [2]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [61]:
import emoji

In [62]:
emoji.EMOJI_UNICODE

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_Is

In [63]:
emoji_dictionary={"0":"\u2764\uFE0F",
                 "1":":baseball:",
                 "2":":grinning_face_with_big_eyes:",
                 "3":":disappointed_face:",
                 "4":":fork_and_knife:",
                 "5":":hundred_points:",
                 "6":":fire:",
                 "7":":face_blowing_a_kiss:",
                 "8":":chestnut:",
                 "9":":flexed_biceps:"}

In [64]:
emoji.emojize(":chestnut:")

'🌰'

In [65]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


#### Step 2: Preparing Custom Dataset

In [228]:
import pandas as pd

In [229]:
import numpy as np

In [230]:
train=pd.read_csv('/Users/shivam_goyal/Downloads/Transfer Learning in NLP/train_emoji.csv',header=None)
test=pd.read_csv('/Users/shivam_goyal/Downloads/Transfer Learning in NLP/test_emoji.csv',header=None)

In [231]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [232]:
data=train.values
print(data.shape)

(132, 4)


In [233]:
X_train=train[0]
Y_train=train[1]

X_test=test[0]
Y_test=test[1]

In [234]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


#### Step 3: Converting sentences in Embeddings

In [235]:
f=open('/Users/shivam_goyal/Downloads/Transfer Learning in NLP/glove.6B.50d.txt',encoding='utf-8')

In [236]:
embeddings_index={}
for line in f:
    values=line.split()
    words=values[0]
    coefs=np.asarray(values[1:],dtype='float')
    embeddings_index[words]=coefs
f.close()


In [237]:
emb_dim=embeddings_index["eat"].shape[0]
print(emb_dim)

50


#### Step 4: Converting sentences into vectors

In [238]:
def embedding_output(X):
    maxLen=10
    embedding_out=np.zeros((X.shape[0],maxLen,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        
        for ij in range(len(X[ix])):
            # Go to every word in the current (ix) sentence
            try:
                embedding_out[ix][ij]=embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij]=np.zeros((50,))
    return embedding_out

embeddings_matrix_train=embedding_output(X_train)
embeddings_matrix_test=embedding_output(X_test)

/Users/shivam_goyal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [239]:
print(X_train[0])
print(len(X_train[0]))

['never', 'talk', 'to', 'me', 'again']
5


In [240]:
print(embeddings_matrix_train.shape)
print(embeddings_matrix_test.shape)

(132, 10, 50)
(56, 10, 50)


In [241]:
print(Y_test.shape)

(56,)


#### Convert Y_test and Y_train to one hot vectors

In [242]:
from keras.utils import to_categorical

In [243]:
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)

In [244]:
print(Y_train.shape,Y_test.shape)

(132, 5) (56, 5)


#### Step 5: Define RNN/LSTM Model

In [245]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [246]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 10, 64)            29440     
_________________________________________________________________
dropout_20 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_21 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 325       
_________________________________________________________________
activation_10 (Activation)   (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________

In [247]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
checkpoint=ModelCheckpoint('best_model.h5',monitor='val_loss',verbose=True,save_best_only=True)
earlystop=EarlyStopping(monitor='val_acc',patience=10)
hist=model.fit(embeddings_matrix_train,Y_train,epochs=250,batch_size=64,shuffle=True,validation_split=0.2)

Epoch 1/250
2/2 [==============================] - 1s 311ms/step - loss: 1.5983 - acc: 0.2476 - val_loss: 1.6310 - val_acc: 0.2593
Epoch 2/250
2/2 [==============================] - 0s 18ms/step - loss: 1.5604 - acc: 0.3619 - val_loss: 1.6333 - val_acc: 0.2222
Epoch 3/250
2/2 [==============================] - 0s 17ms/step - loss: 1.5108 - acc: 0.3619 - val_loss: 1.6421 - val_acc: 0.2222
Epoch 4/250
2/2 [==============================] - 0s 16ms/step - loss: 1.5039 - acc: 0.2857 - val_loss: 1.6561 - val_acc: 0.2222
Epoch 5/250
2/2 [==============================] - 0s 21ms/step - loss: 1.4848 - acc: 0.2952 - val_loss: 1.6681 - val_acc: 0.2222
Epoch 6/250
2/2 [==============================] - 0s 23ms/step - loss: 1.4629 - acc: 0.3429 - val_loss: 1.6736 - val_acc: 0.2222
Epoch 7/250
2/2 [==============================] - 0s 19ms/step - loss: 1.4365 - acc: 0.4000 - val_loss: 1.6794 - val_acc: 0.2222
Epoch 8/250
2/2 [==============================] - 0s 20ms/step - loss: 1.4239 - acc: 0.3

2/2 [==============================] - 0s 22ms/step - loss: 0.0352 - acc: 1.0000 - val_loss: 1.5755 - val_acc: 0.5926
Epoch 64/250
2/2 [==============================] - 0s 20ms/step - loss: 0.0528 - acc: 0.9905 - val_loss: 1.4802 - val_acc: 0.7037
Epoch 65/250
2/2 [==============================] - 0s 20ms/step - loss: 0.0487 - acc: 0.9905 - val_loss: 1.4423 - val_acc: 0.7037
Epoch 66/250
2/2 [==============================] - 0s 23ms/step - loss: 0.0406 - acc: 0.9810 - val_loss: 1.4547 - val_acc: 0.7037
Epoch 67/250
2/2 [==============================] - 0s 32ms/step - loss: 0.0301 - acc: 1.0000 - val_loss: 1.4840 - val_acc: 0.6296
Epoch 68/250
2/2 [==============================] - 0s 15ms/step - loss: 0.0399 - acc: 1.0000 - val_loss: 1.5423 - val_acc: 0.5926
Epoch 69/250
2/2 [==============================] - 0s 14ms/step - loss: 0.0464 - acc: 0.9905 - val_loss: 1.5496 - val_acc: 0.6296
Epoch 70/250
2/2 [==============================] - 0s 24ms/step - loss: 0.0320 - acc: 1.0000 - 

2/2 [==============================] - 0s 17ms/step - loss: 0.0209 - acc: 0.9905 - val_loss: 1.9497 - val_acc: 0.6296
Epoch 126/250
2/2 [==============================] - 0s 18ms/step - loss: 0.0072 - acc: 1.0000 - val_loss: 1.7886 - val_acc: 0.6296
Epoch 127/250
2/2 [==============================] - 0s 17ms/step - loss: 0.0072 - acc: 1.0000 - val_loss: 1.7475 - val_acc: 0.5926
Epoch 128/250
2/2 [==============================] - 0s 23ms/step - loss: 0.0104 - acc: 1.0000 - val_loss: 1.7225 - val_acc: 0.5926
Epoch 129/250
2/2 [==============================] - 0s 21ms/step - loss: 0.0062 - acc: 1.0000 - val_loss: 1.6927 - val_acc: 0.6296
Epoch 130/250
2/2 [==============================] - 0s 20ms/step - loss: 0.0063 - acc: 1.0000 - val_loss: 1.6751 - val_acc: 0.6296
Epoch 131/250
2/2 [==============================] - 0s 16ms/step - loss: 0.0047 - acc: 1.0000 - val_loss: 1.6702 - val_acc: 0.6667
Epoch 132/250
2/2 [==============================] - 0s 29ms/step - loss: 0.0050 - acc: 1.

2/2 [==============================] - 0s 20ms/step - loss: 0.0027 - acc: 1.0000 - val_loss: 1.6883 - val_acc: 0.7037
Epoch 188/250
2/2 [==============================] - 0s 18ms/step - loss: 0.0031 - acc: 1.0000 - val_loss: 1.6923 - val_acc: 0.6667
Epoch 189/250
2/2 [==============================] - 0s 16ms/step - loss: 0.0040 - acc: 1.0000 - val_loss: 1.7002 - val_acc: 0.6667
Epoch 190/250
2/2 [==============================] - 0s 19ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 1.7058 - val_acc: 0.6667
Epoch 191/250
2/2 [==============================] - 0s 18ms/step - loss: 0.0038 - acc: 1.0000 - val_loss: 1.7102 - val_acc: 0.6667
Epoch 192/250
2/2 [==============================] - 0s 18ms/step - loss: 0.0041 - acc: 1.0000 - val_loss: 1.7119 - val_acc: 0.6667
Epoch 193/250
2/2 [==============================] - 0s 17ms/step - loss: 0.0023 - acc: 1.0000 - val_loss: 1.7104 - val_acc: 0.6667
Epoch 194/250
2/2 [==============================] - 0s 14ms/step - loss: 0.0030 - acc: 1.

2/2 [==============================] - 0s 19ms/step - loss: 0.0021 - acc: 1.0000 - val_loss: 1.9723 - val_acc: 0.7037
Epoch 250/250
2/2 [==============================] - 0s 24ms/step - loss: 0.0027 - acc: 1.0000 - val_loss: 1.9747 - val_acc: 0.7037


In [248]:
pred=model.predict_classes(embeddings_matrix_test)

In [249]:
model.evaluate(embeddings_matrix_test,Y_test)

2/2 [==============================] - 0s 4ms/step - loss: 3.1847 - acc: 0.5179


[3.1847212314605713, 0.5178571343421936]

In [133]:
pred=model.predict_classes(embeddings_matrix_test)

In [59]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😞
she got me a present
❤️
😃
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
😞
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious ha ha
😃
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
😃
I love taking breaks
❤️
❤️
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
❤️
Why are you feeling bad
😞
😞
I am upset
😞
😞
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
⚾
valentine day is near
❤️
😃
